In [ ]:
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, ConvLSTM2D, MaxPool3D, UpSampling3D, LeakyReLU
from keras.losses import MeanAbsoluteError, MeanSquaredError
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [ ]:
# Load dataset

# DATASET_FILE = "../dataset/dataset_small_140224.npy"
DATASET_FILE = "../dataset/dataset_smaller_270224.npy"

dataset_original = np.load(DATASET_FILE)
print(dataset_original.shape)
# print(dataset_original.nbytes/1000**3)
#################################
# dataset_original = dataset_original[:2, ...]
#################################

dataset_original = np.float32(dataset_original)

# Add a channel dimension
dataset_original = np.expand_dims(dataset_original, axis=-1)

# Normalize data
print(np.min(dataset_original), np.max(dataset_original), np.mean(dataset_original), np.median(dataset_original), np.var(dataset_original))
dataset_original[dataset_original==-1] = np.nan
scaler = MinMaxScaler()
dataset_original = scaler.fit_transform(dataset_original.reshape(-1, dataset_original.shape[-1])).reshape(dataset_original.shape)
print(np.nanmin(dataset_original), np.nanmax(dataset_original), np.nanmean(dataset_original), np.nanmedian(dataset_original), np.nanvar(dataset_original))
dataset_original[np.isnan(dataset_original)] = -1
print(np.min(dataset_original), np.max(dataset_original), np.mean(dataset_original), np.median(dataset_original), np.var(dataset_original))

# print(dataset_original.shape)
NUM_SEQUENCES = dataset_original.shape[0]
LENGHT_SEQUENCE = dataset_original.shape[1]
WIDTH = dataset_original.shape[2]
HEIGHT = dataset_original.shape[3]
# print(NUM_SEQUENCES, LENGHT_SEQUENCE, WIDTH, HEIGHT)

In [ ]:
WINDOW = 5

# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset_original.shape[0])
np.random.seed(42)
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * dataset_original.shape[0])]
val_index = indexes[int(0.9 * dataset_original.shape[0]) :]
train_dataset = dataset_original[train_index]
val_dataset = dataset_original[val_index]

print(train_dataset.shape)
print(val_dataset.shape)

train_dataset = train_dataset.reshape(-1, WINDOW, train_dataset.shape[2], train_dataset.shape[3], 1)
val_dataset = val_dataset.reshape(-1, WINDOW, val_dataset.shape[2], val_dataset.shape[3], 1)

print(train_dataset.shape)
print(val_dataset.shape)

X_train, y_train = train_dataset[:,:-1,:,:,:], train_dataset[:,-1:,:,:,:]
X_val, y_val = val_dataset[:,:-1,:,:,:], val_dataset[:,-1:,:,:,:]

print(str(X_train.shape) + ", " + str(y_train.shape))
print(str(X_val.shape) + ", " + str(y_val.shape))


In [ ]:
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)


# from tensorflow.python.client import device_lib
# local_device_protos=device_lib.list_local_devices()
# [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']



In [ ]:
batch_size = 16

GPUS = ["GPU:0","GPU:1"]
# GPUS = ["GPU:0"]
strategy = tf.distribute.MirroredStrategy(GPUS)

with strategy.scope():
    input_dimensions = X_train[0].shape

    # Encoder
    input = Input(input_dimensions)
    encoder = ConvLSTM2D(filters=64, kernel_size=(4,4), padding="same",
                        return_sequences=True, activation="relu")(input)
    encoder = MaxPool3D(pool_size=(2,2,2))(encoder)
    encoder = ConvLSTM2D(filters=32, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(encoder)
    encoder = MaxPool3D(pool_size=(2,2,2))(encoder)
    encoder = ConvLSTM2D(filters=32, kernel_size=(2,2), padding="same",
                        return_sequences=True, activation="relu", name='encoder_output')(encoder)

    # Decoder
    decoder = ConvLSTM2D(filters=32, kernel_size=(2,2), padding="same",
                        return_sequences=True, activation="relu")(encoder)
    decoder = UpSampling3D(size=(2,2,2))(decoder)
    decoder = ConvLSTM2D(filters=32, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(decoder)
    decoder = UpSampling3D(size=(2,2,2))(decoder)
    decoder = ConvLSTM2D(filters=64, kernel_size=(4,4), padding="same",
                        return_sequences=True, activation="relu")(decoder)
    decoder = Conv2D(filters=1, kernel_size=(4,4), padding="same",
                        activation=LeakyReLU(), 
                        name='decoder_output')(decoder)

    # Predictor
    predictor = ConvLSTM2D(filters=32, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(encoder)
    predictor = UpSampling3D(size=(2,2,2))(predictor)
    predictor = ConvLSTM2D(filters=64, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(predictor)
    predictor = UpSampling3D(size=(2,2,2))(predictor)
    predictor = ConvLSTM2D(filters=64, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(predictor)
    predictor = Conv2D(filters=1, kernel_size=(3,3), padding="same", 
                        activation=LeakyReLU())(predictor)
    predictor = MaxPool3D(pool_size=(4,1,1), name='predictor_output')(predictor)



    model = Model(inputs=input, 
                  outputs=[decoder, predictor]
                  # outputs=decoder
                )

    model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())

# model.summary()

In [ ]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes


get_model_memory_usage(batch_size, model)

In [ ]:
with strategy.scope():
    # epochs = 4
    epochs = 400
    history = model.fit(
                        X_train, [X_train, y_train],
                        # X_train, X_train, 
                        # validation_data=(X_val, [X_val, y_val]),
                        # validation_data=(X_val, X_val),
                        batch_size=batch_size, epochs=epochs,
                        verbose=1)

model_name = f"./models/_model_{datetime.now():%Y%m%d_%H%M%S}"

model.save(model_name + ".h5", save_format="h5")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Training loss')

ax1.plot(history.history['decoder_output_loss'][2:])
ax1.set_title('Autoencoder loss')
ax1.set_ylabel('loss')
ax1.set_xlabel('epoch')
# plt.legend(['train'], loc='upper left')

ax2.plot(history.history['predictor_output_loss'][2:])
ax2.set_title('Predictor loss')
ax2.set_ylabel('loss')
ax2.set_xlabel('epoch')
# plt.legend(['train'], loc='upper left')
plt.tight_layout()
fig.savefig(model_name + "_training.png")
fig.show()

In [ ]:
# model = load_model("./models/_model_20240229_184755.h5")
model = load_model("./models/v3_model_20240228_170815.h5")

In [ ]:
# model.summary()

In [ ]:
results = model.evaluate(X_val, [X_val, y_val])
print(results)

In [ ]:
# Visualize Encoder-Decoder
example_sequence = X_train[100].copy()
print("example_sequence:", np.min(example_sequence), np.max(example_sequence), np.mean(example_sequence), np.median(example_sequence), np.var(example_sequence))

predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
print("predicted_sequence:", np.min(predicted_sequence), np.max(predicted_sequence), np.mean(predicted_sequence), np.median(predicted_sequence), np.var(predicted_sequence))


color_map = plt.cm.jet
color_map.set_bad(color='black')
fig, axes = plt.subplots(1, 4, figsize=(40, 20))
for idx, ax in enumerate(axes.flat):

    example_frame = example_sequence[idx].copy()
    example_frame[example_frame==-1] = np.nan

    ax.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
    ax.set_title(f"Original Frame {idx + 1}")
    ax.axis("off")

color_map = plt.cm.jet
color_map.set_bad(color='black')
fig, axes = plt.subplots(1, 4, figsize=(40, 20))
for idx, ax in enumerate(axes.flat):
    
    example_frame = example_sequence[idx]
    example_frame_min=np.min(example_frame[example_frame!=0])
    example_frame_max=np.max(example_frame)
    
    predicted_frame = predicted_sequence[0][idx]
    predicted_frame[predicted_frame<0.0] = np.nan
    predicted_frame[predicted_frame<example_frame_min] = np.nan
    predicted_frame[predicted_frame>example_frame_max] = np.nan

    ax.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
    ax.set_title(f"Decoded Frame {idx + 1}")
    ax.axis("off")

In [ ]:
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()
print("example_frame:", np.min(example_frame), np.max(example_frame), np.mean(example_frame), np.median(example_frame), np.var(example_frame))

color_map = plt.cm.jet
color_map.set_bad(color='black')
example_frame[example_frame==-1] = np.nan

plt.clf()
plt.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
plt.title(f"Original Frame")
plt.colorbar(label="Velocity", orientation="horizontal")

In [ ]:
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()
print("example_frame:", np.min(example_frame), np.max(example_frame), np.mean(example_frame), np.median(example_frame), np.var(example_frame))

predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]

print("predicted_frame:", np.min(predicted_frame), np.max(predicted_frame), np.mean(predicted_frame), np.median(predicted_frame), np.var(predicted_frame))

example_frame_min=np.min(example_frame[example_frame!=-1])
example_frame_max=np.max(example_frame)

# print(example_frame_min, example_frame_max, np.mean(example_sequence[0]), np.median(example_sequence[0]), np.var(example_sequence[0]))
# print(np.min(predicted_frame[predicted_frame>-1]), np.max(predicted_frame), np.mean(predicted_frame), np.median(predicted_frame), np.var(predicted_frame))

color_map = plt.cm.jet
color_map.set_bad(color='black')

predicted_frame[predicted_frame<example_frame_min] = np.nan
predicted_frame[predicted_frame>example_frame_max] = np.nan

plt.clf()
plt.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
plt.title(f"Decoded Frame")
plt.colorbar(label="Velocity", orientation="horizontal")


In [ ]:
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]
frame_min=np.min(frame[frame!=-1])
frame_max=np.max(frame)
frame[frame==-1] = np.nan
# predicted_frame[frame==-1] = np.nan


plt.clf()
plt.hist(frame.flatten(), bins='auto')
plt.title("Histogram Encoded Frame")
plt.xlabel('Velocity')
plt.ylabel('Frequency')
plt.show()

# plt.clf()
# plt.hist(predicted_frame.flatten(), bins='auto')
# plt.title("Histogram Decoded Frame")
# plt.xlabel('Velocity')
# plt.ylabel('Frequency')
# plt.show()

predicted_frame[predicted_frame<frame_min] = np.nan
predicted_frame[predicted_frame>frame_max] = np.nan

plt.clf()
plt.hist(predicted_frame.flatten(), bins='auto')
plt.title("Histogram Decoded Frame")
plt.xlabel('Velocity')
plt.ylabel('Frequency')
plt.show()

In [ ]:
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]
frame_min=np.min(frame[frame!=-1])
frame_max=np.max(frame)

frame_data = frame.flatten()
predicted_data = predicted_frame.flatten()

frame_data = np.delete(frame_data, np.where(frame_data == -1))

predicted_data = np.delete(predicted_data, np.where(predicted_data<frame_min))
predicted_data = np.delete(predicted_data, np.where(predicted_data>frame_max))


hist_frame, bin_edges_frame = np.histogram(frame_data, bins='auto')
bin_center_frame = (bin_edges_frame[:-1] + bin_edges_frame[1:])/2
hist_predicted, bin_edges_predicted = np.histogram(predicted_data, bins='auto')
bin_center_predicted = (bin_edges_predicted[:-1] + bin_edges_predicted[1:])/2

plt.figure()
plt.figure(figsize=(20,6))
plt.errorbar(bin_center_frame, hist_frame, yerr=0, fmt='.', color="green")
plt.errorbar(bin_center_predicted, hist_predicted, yerr=0, fmt='|', color="blue")
plt.show()

In [ ]:
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]

# residual = np.abs(frame - predicted_frame)
residual = np.square(frame - predicted_frame)

plt.clf()
plt.imshow(np.squeeze(residual).transpose(), cmap="Grays")
plt.colorbar(label="error", orientation="horizontal")

In [ ]:
# Visualize Encoder-Predict

# example_sequence = X_val[100].copy()
# example_output = y_val[100].copy()
# predicted_frame = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[1][0]
# print(example_sequence.shape)
# print(example_output.shape)
# print(predicted_frame.shape)


color_map = plt.cm.jet
color_map.set_bad(color='black')

fig, axes = plt.subplots(2, 4, figsize=(80, 20))
# Plot the original frames.
for idx, ax in enumerate(axes[0]):

    example_frame = y_val[idx].copy()
    example_frame[example_frame==-1] = np.nan

    ax.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
    ax.set_title(f"Sequence {idx + 1}")
    ax.axis("off")

# Plot the new frames.
for idx, ax in enumerate(axes[1]):

    example_sequence = X_val[idx].copy()
    example_frame = y_val[idx].copy()

    predicted_frame = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[1][0]
    
    predicted_frame[predicted_frame<0.0] = np.nan

    ax.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
    ax.set_title(f"Sequence {idx + 1}")
    ax.axis("off")

In [ ]:
expected_frames = []
predicted_frames = []

example_sequence = dataset_original[20].copy()
# limit = len(sequence)-WINDOW
limit = 20
for i in range(limit):
    i += 100

    j = i+WINDOW
    slice = example_sequence[i:j]
    input = slice[0:WINDOW-1]
    expected_output = slice[WINDOW-1:]
    expected_frames.append(expected_output)
    new_prediction = model.predict(np.expand_dims(input, axis=0), verbose=0)[1][0]
    predicted_frames.append(new_prediction)


color_map = plt.cm.jet
color_map.set_bad(color='black')

fig, axes = plt.subplots(2, limit, figsize=(30*limit, 15*2))
# Plot the original frames.
for idx, ax in enumerate(axes[0]):

    example_frame = expected_frames[idx].copy()
    example_frame[example_frame==-1] = np.nan

    ax.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
    ax.set_title(f"Frame {idx + WINDOW}")
    ax.axis("off")

# Plot the new frames.
for idx, ax in enumerate(axes[1]):

    predicted_frame = predicted_frames[idx]
    predicted_frame[predicted_frame<0.0] = np.nan

    ax.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
    ax.set_title(f"Frame {idx + WINDOW}")
    ax.axis("off")


In [ ]:
# np.set_printoptions(threshold=np.inf)
# np.set_printoptions(linewidth=np.inf)

# WINDOW = 5

# example_sequence = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], dtype=np.float32)

# predicted_sequence_size = example_sequence.size

# predicted_sequence = np.zeros(predicted_sequence_size)

# predicted_sequence[0:WINDOW-1] = example_sequence[0:WINDOW-1]

# print(example_sequence)
# print(predicted_sequence)

# for i in range(predicted_sequence_size-WINDOW+1):

#     input = predicted_sequence[i:i+WINDOW-1]
    
#     #####prediction######
#     predicted_frame = input[-1] + 1
#     #####################

#     predicted_sequence[i+WINDOW-1] = predicted_frame

# print(example_sequence)
# print(predicted_sequence)

In [ ]:
example_sequence = dataset_original[3].copy()

predicted_sequence_size = len(example_sequence)
dim=example_sequence.shape

predicted_sequence = np.zeros(dim)

predicted_sequence[0:WINDOW-1] = example_sequence[0:WINDOW-1]

frame_residual = 0
total_residual = 0

for i in range(predicted_sequence_size-WINDOW+1):
    # input = predicted_sequence[i:i+WINDOW-1]
    input = example_sequence[i:i+WINDOW-1]

    predicted_frame = model.predict(np.expand_dims(input, axis=0), verbose=0)[1][0]

    predicted_sequence[i+WINDOW-1] = predicted_frame

print(example_sequence.shape)
print(predicted_sequence.shape)

In [ ]:
color_map = plt.cm.jet
color_map.set_bad(color='black')

limit=40
fig, axes = plt.subplots(2, limit, figsize=(10*limit, 5*2))
# fig, axes = plt.subplots(limit, 2, figsize=(5*2, 10*limit))

# Plot the original frames.
for idx, ax in enumerate(axes[0]):

    example_frame = example_sequence[idx].copy()
    example_frame[example_frame==-1] = np.nan

    ax.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
    ax.set_title(f"Frame {idx}")
    ax.axis("off")

# Plot the new frames.
for idx, ax in enumerate(axes[1]):

    predicted_frame = predicted_sequence[idx].copy()
    predicted_frame[predicted_frame<0.0] = np.nan

    ax.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
    ax.set_title(f"Frame {idx}")
    ax.axis("off")


In [ ]:
color_map = plt.cm.jet
color_map.set_bad(color='black')

idx = 100

_, axes = plt.subplots(2, 1)

# original
example_frame = example_sequence[idx].copy()
example_frame[example_frame==-1] = np.nan 
axes[0].imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
axes[0].title.set_text('Original')
axes[0].axis("off")


# generated
predicted_frame = predicted_sequence[idx].copy()
predicted_frame[predicted_frame<0.0] = np.nan
axes[1].imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
axes[1].title.set_text('Generated')
axes[1].axis("off")